#dependencies, database, and nested function


##dependencies

In [ ]:
#before running install Capability Classifier.keras (a dependencies) first in content 
from google.colab import files
files.upload()

In [ ]:
pip install tf-models-nightly 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 22.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 117.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.9/488.9 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 89.0 MB/s eta 0:00:00
  

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from datetime import datetime, date, timedelta
from tensorflow.keras import layers
tf.__version__

'2.14.0-dev20230602'

In [ ]:
CC_model=tf.keras.models.load_model('Capability Classifier.keras')

##database (temporary)

In [ ]:
workout_data = pd.DataFrame({'type': ['push up', 'sit up'],
                            'floor': [5, 5],
                            'ceil': [13, 15],
                            'duration': [10, 10]})

##nested function

In [ ]:
def Binary(G):
    if G == 'M':
        return 1
    elif G == 'F':
        return 0

def BMI(H,W):
        return W/((H/100)**2)

def Status(x):
    if x < 18.75:
        return 'Underweight'
    elif x >= 18.75 and x < 25:
        return 'Normal'
    elif x >= 25 and x < 30:
        return 'Overweight'
    elif x >= 30 and  x < 40:
        return 'Obesity'
    elif x >= 40:
        return 'Severe Obesity'

#inputing users data

In [ ]:
Gender = str(input("What's your gender? (M/F): "))
Weight = float(input("What's your weight : "))
Height = float(input("What's your height: "))
Freq_wo = int(input("How frequent would you like your workout to be (3, 4, 5, 6, 7): "))
Day_start = str(input("When would you like to start your workout (today, tomorrow): "))
When_wo = str(input("When will you available for workout (morning, afternoon, night): ")) 

What's your gender? (M/F): M
What's your weight : 81
What's your height: 177
How frequent would you like your workout to be (3, 4, 5, 6, 7): 4
When would you like to start your workout (today, tomorrow): tomorrow
When will you available for workout (morning, afternoon, night): afternoon


In [ ]:
sample = {
    'Binary': Binary(Gender),
    'Weight': Weight,
    'Height': Height,
    'BMI': BMI(Height, Weight),
    'Status': Status(BMI(Height, Weight)),
    'Freq_of_WO': Freq_wo,
    'Day_to_start': Day_start,
    'When_to_wo': When_wo,
}
print(sample)

{'Binary': 1, 'Weight': 81.0, 'Height': 177.0, 'BMI': 25.854639471416256, 'Status': 'Overweight', 'Freq_of_WO': 4, 'Day_to_start': 'tomorrow', 'When_to_wo': 'afternoon'}


#schedule generator

In [ ]:
mod_input = {}

for name, value in sample.items():
    if name in ['Binary', 'Weight', 'Height', 'BMI', 'Status']:
        converted_value = tf.convert_to_tensor([value])
        mod_input[name] = converted_value

predictions = CC_model.predict(mod_input)
prob = tf.nn.sigmoid(predictions[0])

print(
    "The user has a %.8f percent probability "
    "of being considered capable." % (100 * prob))

1/1 [==============================] - 0s 33ms/step
The user has a 9.72207451 percent probability of being considered capable.


In [ ]:
wo_schedule = pd.DataFrame({'exercise.no': range(1, (4*sample.get('Freq_of_WO')+1))})  

#FORMULATE PLAN
wo_schedule['plan'] = None

set_plan = {value: math.ceil(row['floor'] + prob * (row['ceil'] - row['floor']))
             for _, row in workout_data.iterrows()
             for value in [row['type']]}

wo_schedule['plan'] = wo_schedule['plan'].apply(lambda x: set_plan)


#FORMULATE SETS
wo_schedule['sets'] = wo_schedule['exercise.no'].apply(lambda x: 2 if (x-(x % sample.get('Freq_of_WO')))/sample.get('Freq_of_WO') == 0
                                                      else 3 if (x-(x % sample.get('Freq_of_WO')))/sample.get('Freq_of_WO') in [1, 2]
                                                      else 4 if (x-(x % sample.get('Freq_of_WO')))/sample.get('Freq_of_WO') in [3, 4]
                                                      else None)


#FORMULATE DATE
wo_schedule['date'] = wo_schedule['exercise.no'].apply(lambda x: round((x-1)*7/sample.get('Freq_of_WO')) if sample.get('Day_to_start') == "today"
                                                      else round(x*7/sample.get('Freq_of_WO')) if sample.get('Day_to_start') == "tomorrow"
                                                      else None)
wo_schedule['date'] = wo_schedule['date'].apply(lambda x: (date.today() + timedelta(days=x)).strftime("%d-%m-%Y"))


#FORMULATE START TIME
wo_schedule['start'] = None
wo_schedule['start'] = wo_schedule['start'].apply(lambda x: "07:00" if sample.get('When_to_wo') == "morning"
                                                  else "16:00" if sample.get('When_to_wo') == "afternoon"
                                                  else "19:00" if sample.get('When_to_wo') == "night"
                                                  else None)


#FORMULATE END TIME
wo_schedule['end'] = None
for _, row in wo_schedule.iterrows():
    end_time = datetime.strptime(row['start'], "%H:%M")    
    
    for exercise in row['plan']:
        reps = math.ceil(
               workout_data.loc[workout_data['type'] == exercise, 'floor'].values[0] + prob * (
               workout_data.loc[workout_data['type'] == exercise, 'ceil'].values[0] - workout_data.loc[workout_data['type'] == exercise, 'floor'].values[0]))
        duration = workout_data.loc[workout_data['type'] == exercise, 'duration'].values[0]
        sets = row['sets']
        interval = int(duration * sets * reps)+ 30*(sets-1) 
        end_time += timedelta(seconds=interval)
    
    wo_schedule.at[_, 'end'] = end_time.strftime("%H:%M") 

wo_schedule.head(4*sample.get('Freq_of_WO')+1)

,exercise.no,plan,sets,date,start,end
0,1,"{'push up': 6, 'sit up': 6}",2,04-06-2023,16:00,16:05
1,2,"{'push up': 6, 'sit up': 6}",2,06-06-2023,16:00,16:05
2,3,"{'push up': 6, 'sit up': 6}",2,07-06-2023,16:00,16:05
3,4,"{'push up': 6, 'sit up': 6}",3,09-06-2023,16:00,16:08
4,5,"{'push up': 6, 'sit up': 6}",3,11-06-2023,16:00,16:08
5,6,"{'push up': 6, 'sit up': 6}",3,12-06-2023,16:00,16:08
6,7,"{'push up': 6, 'sit up': 6}",3,14-06-2023,16:00,16:08
7,8,"{'push up': 6, 'sit up': 6}",3,16-06-2023,16:00,16:08
8,9,"{'push up': 6, 'sit up': 6}",3,18-06-2023,16:00,16:08
9,10,"{'push up': 6, 'sit up': 6}",3,20-06-2023,16:00,16:08
